## Startup
Primeiramente são iniciados valores pré-definidos para serem usados como parâmetros.

In [ ]:
"""hog"""
orientations = 9
window_dim = [153, 70]
step = [10, 10]
cells_per_block = [6, 3]
pixels_per_cell = [3, 2]
normalize_with = 'L1-sqrt'
visualize = False
desired_confidence = 0.
scale_step = 1.25
sigma = 2.5


Então define-se as funções de carregar imagens, selecionar imagens aleatoriamente, mostrar imagens, e extrair as features de hog.

In [ ]:
import os
from skimage.io import imread
import fnmatch

def load_dir_images(path, fltr):
    img_paths = []
    
    for root, _, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, fltr):
            img_paths.append(os.path.join(root, filename))

    images = []
    for path in img_paths:
        images.append(imread(path))
    return images

In [ ]:
import matplotlib.pyplot as plt
import random as rnd
from operator import itemgetter

def select_random_images(images, ceil):
    indx = rnd.sample(range(0, len(images)), ceil)
    imgs = itemgetter(*indx)(images)
    return imgs, indx

def show(images, title = "Random images plot", ceil = 5, is_gray = False):
    im_number = len(images)
    if im_number > ceil:
        imgs = select_random_images(images, ceil)
    elif im_number == 0:
        return
    else:
        imgs = images
    fig, ax = plt.subplots(1, len(imgs), figsize=(23, 2))
    index = 0
    fig.suptitle(title)
    for image in imgs:
        if is_gray:
            ax[index].imshow(image, cmap=plt.cm.gray)
        else:
            ax[index].imshow(image)
        index += 1
    plt.show()

In [ ]:
from skimage.feature import hog
from operator import itemgetter
from skimage.color import rgb2gray

def extract_features(images, idxs = None):
    fd = []
    imgs = []
    for img in images:
        im = flt.gaussian(img, sigma=(sigma, sigma), truncate=2., multichannel=True)
        if visualize:
            fdi, im = hog(rgb2gray(im), orientations, pixels_per_cell, cells_per_block, normalize_with, visualize)
            fd.append(fdi)
            imgs.append(im)
        else:
            fd.append(hog(rgb2gray(im), orientations, pixels_per_cell, cells_per_block, normalize_with, visualize))
    if not idxs is None and len(imgs) > max(idxs):
        show(itemgetter(*idxs)(imgs), "hog", is_gray=True)
    return fd

In [ ]:
from skimage.transform import resize

pos = load_dir_images("../data/pos/", '*.png')
for i in range(0, len(pos)):
    pos[i] = resize(pos[i], [70, 153])
pos_sample, pos_idx = select_random_images(pos, 5)
show(pos_sample, "Positive examples")
neg = load_dir_images("../data/neg/", '*.png')
for i in range(0, len(neg)):
    neg[i] = resize(neg[i], [70, 153])
neg_sample, neg_idx = select_random_images(neg, 5)
show(neg_sample, "Negative examples")

In [ ]:
pos_fd = extract_features(pos, pos_idx)
neg_fd = extract_features(neg, neg_idx)

In [ ]:
from sklearn.svm import LinearSVC

def train_svm(pos_fd, neg_fd):
    target = []
    features = []
    for fd in pos_fd:
        target.append(1)
        features.append(fd)

    for fd in neg_fd:
        target.append(0)
        features.append(fd)

    machine = LinearSVC()
    machine.fit(features, target)
    return machine

In [ ]:
from collections import namedtuple
Window = namedtuple("Window", "x y pixels is_border is_bottom")

def get_next_point(current, shape):
    x = current.x + step[0]
    bottom = current.is_bottom
    border = False
    if x >= shape[1] or current.is_border:
        x = 0
        y = current.y + step[1]
    elif x + window_dim[0] > shape[1]:
        x = shape[1] - window_dim[0]
        y = current.y
        border = True
    else:
        y = current.y

    if y >= shape[0] or (current.is_bottom and current.is_border):
        raise type("ImageBoundsReached", (Exception, object), {})
    elif y + window_dim[1] > shape[0]:
        y = shape[0] - window_dim[1]
        bottom = True
    return x, y, border, bottom    

In [ ]:
def get_next_window(image, current=None):
    if current is None:
        x, y = 0, 0
        bottom = False
        border = False
    else:
        try:
            x, y, border, bottom = get_next_point(current, image.shape)
        except Exception:
            return None
        #x, y = current.x + step[0], current.y + step[1]
    return Window(x, y, image[y:y+window_dim[1], x:x+window_dim[0]], border, bottom)

In [ ]:
def widen_search(original, scale):
    if scale <= 0:
        return original 
    new_width = original.shape[0]//scale
    new_height = original.shape[1]//scale
    resized = resize(original, [new_width, new_height])
    if np.less_equal(resized.shape[0:2], np.array(window_dim)).any():
        return None
    return resized

In [ ]:
import numpy as np
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import skimage.filters as flt


test = load_dir_images("../data/test/", '*.jpg')

def valor(source):
    classify_svm(machine, test[source], 0, False)

def classify_svm(machine, image, count, verbose=False):
    im = None
    scale = 0
    maximum = 0
    blurred = flt.gaussian(image, sigma=(sigma, sigma), truncate=2., multichannel=True)
    plt.imshow(blurred)
    plt.show()
    while(not (img := widen_search(blurred, scale)) is None):
        print(img.shape)
        while(not (im := get_next_window(img, im)) is None):
            if verbose:
                print("Running with window at ({}, {}) up to ({},{})".format(im.x,
                                                                         im.y,
                                                                         im.pixels.shape[1]+im.x,
                                                                         im.pixels.shape[0]+im.y))
            if visualize:
                result = hog(rgb2gray(im.pixels), orientations, pixels_per_cell, cells_per_block, normalize_with, visualize)
                feat, im_hog = result
                show([im_hog, im.pixels], "hog", 2, True)
            else:
                feat = hog(rgb2gray(im.pixels), orientations, pixels_per_cell, cells_per_block, normalize_with, visualize)
            feat = np.array(feat).reshape(1, (len(feat)))
            prediction = machine.predict(feat)
            confidence = machine.decision_function(feat)
            maximum = confidence if maximum < confidence else maximum
            if prediction == 1 and confidence >= desired_confidence:
                if verbose:
                    print("Found one at ({},{}) up to ({},{})".format(im.x,
                                                                      im.y,
                                                                      im.pixels.shape[1]+im.x,
                                                                      im.pixels.shape[0]+im.y))
                image_filename = "{}.png".format(confidence)
                blurred_filename = "{}_blurred.png".format(confidence)
                plt.imsave(os.path.join("../data/identified/", blurred_filename), im.pixels)
                plt.imsave(os.path.join("../data/identified/", image_filename), widen_search(image, scale)[im.y:im.y+im.pixels.shape[0], im.x:im.x+im.pixels.shape[1]]) 
                count+=1
                #show([im_hog, im.pixels], "hog", 2, True)
                #plt.show()
        scale += scale_step
    print(maximum)
    return count

sources = [i for i in range(0, len(test)-1)]
test_cases = len(test)
machine = train_svm(pos_fd, neg_fd)
selected_test_case = rnd.randint(0, test_cases)
#print("Of {} possible, test case #{} was selected.".format(test_cases, selected_test_case))
#selected_test_case = test[selected_test_case]
#plt.imshow(selected_test_case)
#plt.show()
show(test, "test cases", len(test))
val = interact_manual(valor, source=sources)